# Parsing of MP3 Audio Files (using pydub)

### Author: John Sugden
### Edited by: Langchen Fan
### Last Updated: 06/30/2022

In [ ]:
# note, you will need to install ffmpeg in order to run this project ('brew install ffmpeg'); see https://github.com/jiaaro/pydub#dependencies
# documentation suggests labav as an alternative, but the library has been depreciated and is no longer maintained

from pydub import AudioSegment

## read in tracks
fox = AudioSegment.from_mp3("unprocessed_audio/FOXSPORTS_06-09-2022_09-00-00_PM_28518.mp3")
# ipri = AudioSegment.from_mp3("unprocessed_audio/IPRI-FL_06-09-2022_09-00-00_PM_28516.mp3")
# kbpi = AudioSegment.from_mp3("unprocessed_audio/KBPI-FM_06-09-2022_11-00-00_PM_28514.mp3")
# kdge = AudioSegment.from_mp3("unprocessed_audio/KDGE-FM_06-09-2022_10-00-00_PM_28509.mp3")
# kdmx = AudioSegment.from_mp3("unprocessed_audio/KDMX-FM_06-09-2022_10-00-00_PM_28508.mp3")
# kegl = AudioSegment.from_mp3("unprocessed_audio/KEGL-FM_06-09-2022_10-00-00_PM_28507.mp3")
# khks = AudioSegment.from_mp3("unprocessed_audio/KHKS-FM_06-09-2022_10-00-00_PM_28505.mp3")
# kiis = AudioSegment.from_mp3("unprocessed_audio/KIIS-FM_06-10-2022_12-00-00_AM_28489.mp3")
# koda = AudioSegment.from_mp3("unprocessed_audio/KODA-FM_06-09-2022_10-00-00_PM_28500.mp3")
# kwbl = AudioSegment.from_mp3("unprocessed_audio/KWBL-FM_06-09-2022_11-00-00_PM_28515.mp3")
# kyld = AudioSegment.from_mp3("unprocessed_audio/KYLD-FM_06-10-2022_12-00-00_AM_28512.mp3")
# kzps = AudioSegment.from_mp3("unprocessed_audio/KZPS-FM_06-09-2022_10-00-00_PM_28506.mp3")
# pnml = AudioSegment.from_mp3("unprocessed_audio/PNML_06-09-2022_09-00-00_PM_28815.mp3")
# wash = AudioSegment.from_mp3("unprocessed_audio/WASH-FM_06-09-2022_09-00-00_PM_28503.mp3")
# waxq = AudioSegment.from_mp3("unprocessed_audio/WAXQ-FM_06-09-2022_09-00-00_PM_28490.mp3")
# whlh = AudioSegment.from_mp3("unprocessed_audio/WHLH-FM_06-09-2022_10-00-00_PM_28517.mp3")
# whzt = AudioSegment.from_mp3("unprocessed_audio/WHTZ-FM_06-09-2022_09-00-00_PM_28491.mp3")
# wrda = AudioSegment.from_mp3("unprocessed_audio/WRDA-FM_06-09-2022_09-00-00_PM_28501.mp3")
# wwdc = AudioSegment.from_mp3("unprocessed_audio/WWDC-FM_06-09-2022_09-00-00_PM_28504.mp3")
# wwpr = AudioSegment.from_mp3("unprocessed_audio/WWPR-FM_06-09-2022_09-00-00_PM_28513.mp3")

## combine (for looping purposes)
# station_list = [fox, ipri, kbpi, kdge, kdmx, kegl, khks, kiis, koda, kwbl, kyld, kzps, pnml, wash, waxq, whlh, whzt, wrda, wwdc, wwpr]
station_names = ['fox', 'ipri', 'kbpi', 'kdge', 'kdmx', 'kegl', 'khks', 'kiis', 'koda', 'kwbl', 'kyld', 'kzps', 'pnml', 'wash', 'waxq', 'whlh', 'whzt', 'wrda', 'wwdc', 'wwpr']

In [ ]:
# check some recording characteristics
from scipy import signal
import matplotlib.pyplot as plt
plt.plot(fox)

In [114]:
## create minute-long noise tracks - tones, buzzes, pops, static
static = AudioSegment.from_wav("noise/audiocheck.net_violetnoise.wav") - 10 # lower 10 decibels
full_static = static * 6
full_static = full_static[0:59999] ## make sure 60 seconds long
full_static.export("simulated_static.wav", format="wav")

buzz = AudioSegment.from_wav("noise/RiserDetune_SP_238_01.wav") - 15 ## lower by 15 decibels
full_buzz = buzz[0:4999] * 12
full_buzz = full_buzz[0:59999]
full_buzz.export("simulated_buzz.wav", format="wav")

glitch = AudioSegment.from_wav("noise/100_ D_GlitchDroneFx_849.wav") - 25 ## lower by 25 decibels
full_glitch = glitch[1000:3999] * 20
full_glitch = full_glitch[0:59999]
full_glitch.export("simulated_glitch.wav", format="wav")

<_io.BufferedRandom name='simulated_dual.wav'>

In [116]:
## read in noise
simulated_static = AudioSegment.from_wav("simulated_static.wav")
simulated_buzz   = AudioSegment.from_wav("simulated_buzz.wav")
simulated_glitch = AudioSegment.from_wav("simulated_glitch.wav")

In [140]:
# parsing larger audio into one-minute samples
## note, pydub does things in miliseconds
min = 60*1000

## loop through stations
for x in range(20):
    audio_track = station_list[x]

    ## cut 90 minute audio track into 90 one-minute samples
    for y in range(90):

        # parse
        audio_start = min*y
        audio_stop = (min*(y+1)) - 1
        sample = audio_track[audio_start:audio_stop]

        # overlay static
        if x == 16:
            sample = sample.overlay(simulated_static, position=0)

        # overlay buzz
        if x == 17:
            sample = sample.overlay(simulated_buzz, position=0)

        # overlay glitch
        if x == 18:
            sample = sample.overlay(simulated_glitch, position=0)

        # overlay dual track
        if x == 19:
            simulated_dual = kiis[audio_start:audio_stop]
            sample = sample.overlay(simulated_dual, position=0)

        # file name formatting
        trailing_zero = ''
        if y < 9:
            trailing_zero = '0'
        track_name = 'samples/' + station_names[x] + '_' + trailing_zero + str(y+1) + '.wav'

        # save out
        sample.export(track_name, format="wav")



'samples/ipri_2.mp3'